<a href="https://colab.research.google.com/github/yang1325/AI_selfpraticenote/blob/main/tensorflow_covid/tensorflow_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#載入所需模組
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import numpy as np
import pandas as pd
from numpy import load
from numpy import save
import random
import os
import gc

In [4]:
#事先將資料上傳到google drive 然後用google colab 連接
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
labels = ['Normal', 'Viral Pneumonia', 'COVID', 'Lung_Opacity']
labels

['Normal', 'Viral Pneumonia', 'COVID', 'Lung_Opacity']

In [6]:
import cv2
import numpy as np
number = [0]
tem = 0
for i in range(len(labels)):
  path = "/content/gdrive/My Drive/Covid_data_merge/" + labels[i]
  tem = tem + len(os.listdir(path))
  number += [len(os.listdir(path))]
print(number)
data = np.zeros((tem,150,150),dtype = int)
y = np.zeros((tem,len(labels)),dtype = int)
tem = 0
for i in range(len(labels)):
  print(labels[i])
  path = "/content/gdrive/My Drive/Covid_data_merge/" + labels[i]
  os.chdir(path)
  pic_list = os.listdir(path)
  tem += number[i]
  for j in range(len(pic_list)):
    y[tem + j,i] = 1
    data[tem + j,:,:] = cv2.imread(pic_list[j], cv2.IMREAD_GRAYSCALE)

[0, 10192, 1345, 3616, 6012]
Normal
Viral Pneumonia
COVID
Lung_Opacity


In [7]:
gc.collect()
n = sum(number)
random_index = np.array(random.sample(range(0,n),n))
data = data[random_index,:,:]
y = y[random_index,:]
gc.collect()

0

In [8]:
test_index = np.array(random.sample(range(0,n), int(n * 0.2)))
train_index = np.ones(n, dtype=bool)
train_index[test_index] = False
test_label = y[test_index,:]
train_label = y[train_index,:]

In [9]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(64, (2, 2), activation='relu', input_shape=(150, 150, 1)))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(128, (2, 2), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(256, (2, 2), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(512, (2, 2), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(1024, (2, 2), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Flatten())
cnn_model.add(tf.keras.layers.Dropout(0.2))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(layers.Dense(4, activation="softmax"))
cnn_model.summary()
gc.collect()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 149, 149, 64)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 73, 73, 128)       32896     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 35, 35, 256)       131328    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 256)      0

0

In [ ]:
checkpoint_path = "/content/gdrive/My Drive/Covid_data_merge/trained/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cnn_model.load_weights(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
        save_weights_only=True
        ,verbose=1)

cnn_model.compile(optimizer="Adam",
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

history = cnn_model.fit(data[train_index,:,:], train_label, epochs=30,batch_size = 1 , 
                    validation_data=(data[test_index,:,:], test_label),
                    callbacks=[cp_callback])

Epoch 1/30
16932/16932 [==============================] - ETA: 0s - loss: 0.7496 - accuracy: 0.7035
Epoch 1: saving model to /content/gdrive/My Drive/Covid_data_merge/trained/cp.ckpt
16932/16932 [==============================] - 2571s 152ms/step - loss: 0.7496 - accuracy: 0.7035 - val_loss: 0.5459 - val_accuracy: 0.7964
Epoch 2/30
16932/16932 [==============================] - ETA: 0s - loss: 0.7619 - accuracy: 0.6974
Epoch 2: saving model to /content/gdrive/My Drive/Covid_data_merge/trained/cp.ckpt
16932/16932 [==============================] - 2568s 152ms/step - loss: 0.7619 - accuracy: 0.6974 - val_loss: 0.5283 - val_accuracy: 0.8068
Epoch 3/30
16932/16932 [==============================] - ETA: 0s - loss: 0.7424 - accuracy: 0.7127
Epoch 3: saving model to /content/gdrive/My Drive/Covid_data_merge/trained/cp.ckpt
16932/16932 [==============================] - 2577s 152ms/step - loss: 0.7424 - accuracy: 0.7127 - val_loss: 0.5069 - val_accuracy: 0.8186
Epoch 4/30
16932/16932 [=======

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
def show_grey(x):
    y = np.zeros((299,299,3))
    y[:,:,0] = x
    y[:,:,1] = x
    y[:,:,2] = x
    return y.astype(int)
import matplotlib.pyplot as plt
plt.imshow(show_grey(data[0,:,:]))
print(os.listdir("/content/gdrive/My Drive/Covid_data/" + labels[0])[0])